# <center>**Data Augmentation**</center>

In [1]:
import os
import cv2
import numpy as np
import albumentations as A

from collections import Counter

In [2]:
# Define the paths of the train labels and the train images
train_images_path = "datasets/weapon-detection-16/train/images"
train_labels_path = "datasets/weapon-detection-16/train/labels"

In [ ]:
# Define the augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5), # flip horizontally with 50% chance
    A.VerticalFlip(p=0.2), # flip vertically with 20% chance
    A.Rotate(limit=20, p=0.5), rotate randomly with a limit of 20 degrees
    A.RandomBrightnessContrast(p=0.5), # apply random brightness and contrast with 50% chance
    A.GaussianBlur(p=0.3), # apply gaussian blur with 30% chance to reduce noise
    A.Resize(640, 640), # resize images back to 640x640 to maintain yolov8 size
], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

In [4]:
# Function to read YOLO labels
def read_yolo_labels(label_path):
    with open(label_path, "r") as f:
        labels = [line.strip().split() for line in f.readlines()]
    return labels

In [5]:
# Function to write YOLO labels
def write_yolo_labels(label_path, labels):
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")

In [6]:
# Get all images and labels
image_files = [f for f in os.listdir(train_images_path) if f.endswith(".jpg") or f.endswith(".png")]

In [7]:
# Augment only images that contain ONLY pistols (Class 0)
augmentation_count = 3  # Number of times to augment each pistol-only image

In [8]:
for img_file in image_files:
    label_file = img_file.replace(".jpg", ".txt").replace(".png", ".txt")
    label_path = os.path.join(train_labels_path, label_file)

    if not os.path.exists(label_path):
        continue

    labels = read_yolo_labels(label_path)

    # Check if the image contains only pistols (Class 0) and no heavy guns (Class 1)
    class_ids = {int(line[0]) for line in labels}  # Extract unique class IDs from labels
    if class_ids != {0}:  # If there's anything other than class 0, skip the image
        continue

    img_path = os.path.join(train_images_path, img_file)
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    # Convert YOLO bbox to pixel coordinates
    bbox_list = []
    class_labels = []
    for line in labels:
        class_id, x, y, w, h = map(float, line)
        bbox_list.append([x, y, w, h])
        class_labels.append(int(class_id))

    # Generate multiple augmented images per pistol-only image
    for i in range(augmentation_count):
        augmented = transform(image=img, bboxes=bbox_list, class_labels=class_labels)
        augmented_img = augmented["image"]
        augmented_bboxes = augmented["bboxes"]

        # Save augmented image
        aug_img_name = f"aug_{i}_{img_file}"
        aug_img_path = os.path.join(train_images_path, aug_img_name)
        cv2.imwrite(aug_img_path, augmented_img)

        # Convert back to YOLO format and save labels
        final_labels = [[class_labels[idx]] + list(bbox) for idx, bbox in enumerate(augmented_bboxes)]
        aug_label_name = f"aug_{i}_{label_file}"
        aug_label_path = os.path.join(train_labels_path, aug_label_name)
        write_yolo_labels(aug_label_path, final_labels)

print('Data augmentation done for pistol- only images!')

Data augmentation done for pistol- only images!


In [9]:
def count_classes_in_yolo_labels(directory):
    class_counts = Counter()
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r") as file:
                for line in file:
                    class_id = line.split()[0]  # Extract the first value (class ID)
                    class_counts[class_id] += 1

    return class_counts

In [10]:
train_dir = "datasets/weapon-detection-16/train/labels"
train_class_counts = count_classes_in_yolo_labels(train_dir)
print("Class counts in train labels:")
print(train_class_counts)

Class counts in train labels:
Counter({'1': 10659, '0': 10222})


In [11]:
validation_dir = "datasets/weapon-detection-16/valid/labels"
validation_class_counts = count_classes_in_yolo_labels(validation_dir)
print("\nClass counts in validation labels:")
print(validation_class_counts)


Class counts in validation labels:
Counter({'1': 919, '0': 396})
